In [3]:
# ==============================================
# Cell 1 — Rules & tiny helpers (super simple)
# ==============================================
# Minimal one-off creation of rules.json (no helpers needed)
from pathlib import Path
import json

In [4]:

DEFAULT_RULES = {
  "report_card_lost": {
    "keywords": ["lost my card", "card stolen", "card missing", "분실", "도난"],
    "args": ["user_id"]
  },
  "check_card_delivery": {
    "keywords": ["track my card", "where is my card", "card not arrived", "배송", "언제 도착"],
    "args": ["card_id"]
  },
  "request_new_card": {
    "keywords": ["order new card", "replace my card", "new card", "재발급"],
    "args": ["user_id"]
  }
}

In [5]:
Path("rules.json").write_text(json.dumps(DEFAULT_RULES, ensure_ascii=False, indent=2), encoding="utf-8")
print("Created rules.json")

Created rules.json


In [6]:
# Load the rules you just created
from pathlib import Path
import json

In [7]:
RULES_PATH = Path("rules.json")
RULES = json.loads(RULES_PATH.read_text(encoding="utf-8"))

THRESHOLD = 0.2  # you can try 0.3 later
print("intents:", list(RULES.keys()))


intents: ['report_card_lost', 'check_card_delivery', 'request_new_card']


In [8]:
# Mock banking functions — replace with real APIs later
import time

In [9]:
def report_card_lost(user_id: str):
    ticket_id = f"T{int(time.time())%100000:05d}"
    return {"ok": True, "ticket_id": ticket_id, "user_id": user_id}

def check_card_delivery(card_id: str):
    return {"ok": True, "card_id": card_id, "carrier": "UPS",
            "eta": "3-5 business days", "last_update": "out for delivery"}

def request_new_card(user_id: str):
    order_id = f"O{int(time.time())%100000:05d}"
    return {"ok": True, "order_id": order_id, "user_id": user_id}


In [10]:
#Intent detection (super simple) + routing

In [11]:
# Naive keyword matcher + function routing
import re

In [12]:
def normalize_text(s: str) -> str:
    s = s.lower()
    s = re.sub(r"[^0-9a-z가-힣\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def predict_intent(text: str, rules: dict) -> tuple[str, float]:
    t = " " + normalize_text(text) + " "
    best = ("nlu_fallback", 0.0)
    for intent, cfg in rules.items():
        kws = cfg.get("keywords", [])
        hits = sum((" "+normalize_text(kw)+" ") in t for kw in kws)
        conf = min(1.0, hits / max(1, len(kws))) if kws else 0.0
        if conf > best[1]:
            best = (intent, conf)
    return best

In [13]:
def run_pipeline(text: str, user_id: str | None = None, card_id: str | None = None):
    intent, conf = predict_intent(text, RULES)
    if conf < THRESHOLD:
        return {"message": "Sorry, I couldn't understand. Could you rephrase?",
                "intent": "nlu_fallback", "confidence": conf,
                "called": None, "result": None}

    if intent == "report_card_lost":
        if not user_id:
            return {"message": "Please provide your user_id.",
                    "intent": intent, "confidence": conf, "called": None, "result": None}
        res = report_card_lost(user_id)
        return {"message": f"I've created a lost-card ticket: {res['ticket_id']}.",
                "intent": intent, "confidence": conf, "called": "report_card_lost", "result": res}

    if intent == "check_card_delivery":
        if not card_id:
            return {"message": "Please provide your card_id.",
                    "intent": intent, "confidence": conf, "called": None, "result": None}
        res = check_card_delivery(card_id)
        return {"message": f"Your card is with {res['carrier']}, ETA {res['eta']} (status: {res['last_update']}).",
                "intent": intent, "confidence": conf, "called": "check_card_delivery", "result": res}

    if intent == "request_new_card":
        if not user_id:
            return {"message": "Please provide your user_id.",
                    "intent": intent, "confidence": conf, "called": None, "result": None}
        res = request_new_card(user_id)
        return {"message": f"I've placed a new card order: {res['order_id']}.",
                "intent": intent, "confidence": conf, "called": "request_new_card", "result": res}

    return {"message": "Unhandled intent. Sorry!",
            "intent": intent, "confidence": conf, "called": None, "result": None}

In [14]:
#4 — Tiny REPL-style helper

In [15]:
# Ask helper; logs labels for self-improvement later
EVAL_ROWS = []

def ask(text: str, user_id: str | None = None, card_id: str | None = None, gold_intent: str | None = None):
    out = run_pipeline(text, user_id=user_id, card_id=card_id)
    print("USER :", text)
    print("ASSIST:", out["message"], f"(intent={out['intent']}, conf={out['confidence']:.2f})")
    if gold_intent:
        EVAL_ROWS.append({"text": text, "label": gold_intent})
        print(f"[logged gold={gold_intent}]")
    return out

In [16]:
#Quick smoke test

In [17]:
ask("I lost my card yesterday", user_id="U1001", gold_intent="report_card_lost")
ask("Where is my card? It hasn't arrived.", card_id="C5555", gold_intent="check_card_delivery")
ask("Please order a new card", user_id="U1001", gold_intent="request_new_card")

USER : I lost my card yesterday
ASSIST: I've created a lost-card ticket: T12851. (intent=report_card_lost, conf=0.20)
[logged gold=report_card_lost]
USER : Where is my card? It hasn't arrived.
ASSIST: Your card is with UPS, ETA 3-5 business days (status: out for delivery). (intent=check_card_delivery, conf=0.20)
[logged gold=check_card_delivery]
USER : Please order a new card
ASSIST: I've placed a new card order: O12851. (intent=request_new_card, conf=0.25)
[logged gold=request_new_card]


{'message': "I've placed a new card order: O12851.",
 'intent': 'request_new_card',
 'confidence': 0.25,
 'called': 'request_new_card',
 'result': {'ok': True, 'order_id': 'O12851', 'user_id': 'U1001'}}

In [18]:
ask("Please order a new card", user_id="U1001", gold_intent="request_new_card")

USER : Please order a new card
ASSIST: I've placed a new card order: O12938. (intent=request_new_card, conf=0.25)
[logged gold=request_new_card]


{'message': "I've placed a new card order: O12938.",
 'intent': 'request_new_card',
 'confidence': 0.25,
 'called': 'request_new_card',
 'result': {'ok': True, 'order_id': 'O12938', 'user_id': 'U1001'}}

In [19]:
ask("I lost my card yesterday", user_id="U1001", gold_intent="report_card_lost")
ask("Where is my card? It hasn't arrived.", card_id="C5555", gold_intent="check_card_delivery")

USER : I lost my card yesterday
ASSIST: I've created a lost-card ticket: T12957. (intent=report_card_lost, conf=0.20)
[logged gold=report_card_lost]
USER : Where is my card? It hasn't arrived.
ASSIST: Your card is with UPS, ETA 3-5 business days (status: out for delivery). (intent=check_card_delivery, conf=0.20)
[logged gold=check_card_delivery]


{'message': 'Your card is with UPS, ETA 3-5 business days (status: out for delivery).',
 'intent': 'check_card_delivery',
 'confidence': 0.2,
 'called': 'check_card_delivery',
 'result': {'ok': True,
  'card_id': 'C5555',
  'carrier': 'UPS',
  'eta': '3-5 business days',
  'last_update': 'out for delivery'}}

In [20]:
improve_from_logs(apply=False, topk=3)  # 제안만 미리 보기
improve_from_logs(apply=True,  topk=3)  # 룰에 반영 + 재평가
save_rules(RULES)                       # 업데이트된 rules.json 저장


NameError: name 'improve_from_logs' is not defined